In [110]:
import pandas as pd
from pandasql import sqldf

In [111]:
#Utilizaremos essa função para rodar a consulta com a query 'q'
pysqldf = lambda q: sqldf(q, globals())

In [112]:
#Carregamos as bases que serão utilziadas
df_VRA = pd.read_csv('./airlines_VRA.csv', dtype={'icao_empresa_aerea':'str','numero_voo': 'str','codigo_autorizacao':'str'})
df_AIR_CIA = pd.read_csv('./airlines_AIR_CIA.csv')
df_aerodromos = pd.read_csv('./airlines_aerodromos_VRA.csv')

In [121]:
q = """WITH rotas_por_cia AS (SELECT icao_empresa_aerea,
       icao_aerodromo_origem,
       icao_aerodromo_destino
FROM (SELECT icao_empresa_aerea,
       icao_aerodromo_origem, 
       icao_aerodromo_destino,
       Rank() OVER (PARTITION BY icao_empresa_aerea ORDER BY qntd_viagens DESC) AS rank_viagens
FROM   (SELECT icao_empresa_aerea,
               icao_aerodromo_origem,
               icao_aerodromo_destino,
               Count(*) AS qntd_viagens
        FROM   df_VRA
        GROUP  BY icao_empresa_aerea, icao_aerodromo_origem, icao_aerodromo_destino) T) R
        WHERE rank_viagens = 1),
    
tabela_razao_social AS (SELECT  C.razao_social, A.icao_empresa_aerea, A.icao_aerodromo_origem as icao_origem_rota, A.icao_aerodromo_destino as icao_destino_rota
FROM rotas_por_cia A INNER JOIN df_AIR_CIA C 
ON A.icao_empresa_aerea = C.icao)

SELECT R.razao_social, S.name as nome_aeroporto_origem, R.icao_origem_rota, S.state as estado_origem, F.name as nome_aeroporto_destino, R.icao_destino_rota, F.state as estado_destino
FROM tabela_razao_social R INNER JOIN df_aerodromos S 
ON R.icao_origem_rota = S.icao 
INNER JOIN df_aerodromos F 
ON R.icao_destino_rota = F.icao;"""


df = pysqldf(q)
df

,razao_social,nome_aeroporto_origem,icao_origem_rota,estado_origem,nome_aeroporto_destino,icao_destino_rota,estado_destino
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,Mariscal Sucre International Airport,SEQM,Pichincha,Miami International Airport,KMIA,Florida
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,Hercílio Luz International Airport,SBFL,Santa Catarina,Caçador Airport,SBCD,Santa Catarina
2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,Marechal Rondon International Airport,SBCY,Mato Grosso,Juína Airport,SWJN,State of Mato Grosso
3,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),Coari Airport,SWKO,State of Amazonas,Eduardo Gomes International Airport,SBEG,Amazonas
4,AZUL LINHAS AÉREAS BRASILEIRAS S/A,Viracopos/Campinas International Airport,SBKP,São Paulo,Santos Dumont Airport,SBRJ,Rio de Janeiro
5,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,Santos Dumont Airport,SBRJ,Rio de Janeiro,São Paulo–Congonhas Airport,SBSP,São Paulo
6,MODERN TRANSPORTE AEREO DE CARGA S.A,Viracopos/Campinas International Airport,SBKP,São Paulo,Brasília International Airport (Presidente J. ...,SBBR,Distrito Federal
7,PASSAREDO TRANSPORTES AÉREOS S.A.,Porto Seguro Airport,SBPS,Bahia,Deputado Luís Eduardo Magalhães International ...,SBSV,Bahia
8,SIDERAL LINHAS AÉREAS LTDA.,São Paulo–Guarulhos International Airport,SBGR,São Paulo,Recife/Guararapes–Gilberto Freyre Internationa...,SBRF,Pernambuco
9,TAM LINHAS AÉREAS S.A.,São Paulo–Congonhas Airport,SBSP,São Paulo,Santos Dumont Airport,SBRJ,Rio de Janeiro


In [108]:
df.to_csv('airlines_SQL_view_aerodromos.csv', index=False, encoding='utf-8-sig')